In [1]:
from sklearn.ensemble import RandomForestClassifier
import deepchem as dc
import numpy as np
import pandas as pd
import tempfile
import chemprop
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras import backend as K
from tensorflow.keras import initializers
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from sklearn.linear_model import LogisticRegression
import xgboost
import os
from sklearn.svm import SVC

2022-11-07 12:42:36.682055: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-07 12:42:36.794449: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-07 12:42:36.794469: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-11-07 12:42:36.816463: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-07 12:42:37.354917: W tensorflow/stream_executor/platform/de

In [2]:
AUC_ROC=[]
model=[]
def list_auc(list_f,list_m):
    main_all_feature=pd.DataFrame()
    for feature in list_f:
        for m in list_m:
            read_main=pd.read_csv('pred_'+m+'_main_set_'+feature+'.csv')
            df_main=pd.DataFrame(read_main)
            pred1=[]
            for p in df_main['pred']:
                if p<=0.5:
                    p=0
                else:
                    p=1
                pred1.append(p)
            main_all_feature.insert(loc=len(main_all_feature.columns),column=m+feature,value=pred1)


    external_all_feature=pd.DataFrame()
    for feature in list_f:
        for m in list_m:
            read_main=pd.read_csv('pred_'+m+'_external_set_'+feature+'.csv')
            df_main=pd.DataFrame(read_main)
            pred1=[]
            for p in df_main['pred']:
                if p<=0.5:
                    p=0
                else:
                    p=1
                pred1.append(p)
            external_all_feature.insert(loc=len(external_all_feature.columns),column=m+feature,value=pred1)
    
    read_m = pd.read_csv("main_set_plus_AtomPairFP.csv")
    df_m = pd.DataFrame(read_m)
    a1 = df_m['active']
    c1=[]
    for b1 in a1:
        if b1==1:
            b1=0
        else:
            b1=1
        c1.append(b1)

    y_m=pd.DataFrame()
    y_m.insert(loc=0, column='active', value=a1)
    y_m.insert(loc=1, column='inactive', value=c1)
    X_m=main_all_feature

    read_e = pd.read_csv("external_set_plus_AtomPairFP.csv")
    df_e = pd.DataFrame(read_e)
    a2 = df_e['active']
    c1=[]
    for b1 in a2:
        if b1==1:
            b1=0
        else:
            b1=1
        c1.append(b1)

    y_e=pd.DataFrame()
    y_e.insert(loc=0, column='active', value=a2)
    y_e.insert(loc=1, column='inactive', value=c1)
    X_e=external_all_feature

    batch_size = 8192
    num_classes = 2
    model = Sequential()
    model.add(Dense(800,
                    activation='relu',
                    input_dim=len(X_m.columns),
                    kernel_initializer = initializers.RandomNormal(stddev=0.02),
                    bias_initializer = initializers.RandomNormal(mean=1.),
                    kernel_regularizer = regularizers.l2(1e-1)))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))
    opt = keras.optimizers.Adam(learning_rate=0.001)

    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=opt,
                  metrics=['AUC'])
    model.fit(X_m, y_m,
              batch_size=batch_size,
              epochs=100)
    score = model.evaluate(X_e, y_e, verbose=0)
    print(score[1])
    AUC_ROC.append(score[1])
    model=''

In [3]:
m='all_model'
a=['AtomPairFP','Connectivity','Charge',
       'Kappa','MAP4','MOE','MorganFP','Topology']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

2022-11-07 12:42:41.689565: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-07 12:42:41.690590: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcufft.so.10'; dlerror: libcufft.so.10: cannot open shared object file: No such file or directory
2022-11-07 12:42:41.690686: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcurand.so.10'; dlerror: libcurand.so.10: cannot open shared object file: No such file or directory
2022-11-07 12:42:41.690768: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcusolver.so.11'; dlerror: libcusolver.so.11: cannot open shared object file: No such file or directory
2022-11-07 12:42:41.690854: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could no

Epoch 1/100
1/1 [==============================] - 0s 392ms/step - loss: 2.5256 - auc: 0.3535
Epoch 2/100
1/1 [==============================] - 0s 25ms/step - loss: 1.6785 - auc: 0.7828
Epoch 3/100
1/1 [==============================] - 0s 32ms/step - loss: 1.7040 - auc: 0.8507
Epoch 4/100
1/1 [==============================] - 0s 33ms/step - loss: 1.7804 - auc: 0.8702
Epoch 5/100
1/1 [==============================] - 0s 29ms/step - loss: 1.8018 - auc: 0.8737
Epoch 6/100
1/1 [==============================] - 0s 33ms/step - loss: 1.7289 - auc: 0.8810
Epoch 7/100
1/1 [==============================] - 0s 31ms/step - loss: 1.5866 - auc: 0.8946
Epoch 8/100
1/1 [==============================] - 0s 33ms/step - loss: 1.4539 - auc: 0.8978
Epoch 9/100
1/1 [==============================] - 0s 34ms/step - loss: 1.3036 - auc: 0.9116
Epoch 10/100
1/1 [==============================] - 0s 29ms/step - loss: 1.1952 - auc: 0.9159
Epoch 11/100
1/1 [==============================] - 0s 31ms/step - l

1/1 [==============================] - 0s 30ms/step - loss: 0.2395 - auc: 0.9911
Epoch 89/100
1/1 [==============================] - 0s 23ms/step - loss: 0.2512 - auc: 0.9883
Epoch 90/100
1/1 [==============================] - 0s 30ms/step - loss: 0.2459 - auc: 0.9895
Epoch 91/100
1/1 [==============================] - 0s 29ms/step - loss: 0.2367 - auc: 0.9917
Epoch 92/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2340 - auc: 0.9914
Epoch 93/100
1/1 [==============================] - 0s 29ms/step - loss: 0.2373 - auc: 0.9915
Epoch 94/100
1/1 [==============================] - 0s 34ms/step - loss: 0.2389 - auc: 0.9905
Epoch 95/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2332 - auc: 0.9917
Epoch 96/100
1/1 [==============================] - 0s 30ms/step - loss: 0.2315 - auc: 0.9918
Epoch 97/100
1/1 [==============================] - 0s 28ms/step - loss: 0.2382 - auc: 0.9898
Epoch 98/100
1/1 [==============================] - 0s 28ms/step - loss: 

In [4]:
m='remove_AtomPairFP'
a=['Connectivity','Charge',
       'Kappa','MAP4','MOE','MorganFP','Topology']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100
1/1 [==============================] - 0s 346ms/step - loss: 2.9166 - auc: 0.2572
Epoch 2/100
1/1 [==============================] - 0s 35ms/step - loss: 1.7772 - auc: 0.6157
Epoch 3/100
1/1 [==============================] - 0s 33ms/step - loss: 1.5440 - auc: 0.8291
Epoch 4/100
1/1 [==============================] - 0s 33ms/step - loss: 1.6870 - auc: 0.8521
Epoch 5/100
1/1 [==============================] - 0s 32ms/step - loss: 1.7894 - auc: 0.8593
Epoch 6/100
1/1 [==============================] - 0s 36ms/step - loss: 1.8107 - auc: 0.8604
Epoch 7/100
1/1 [==============================] - 0s 25ms/step - loss: 1.7780 - auc: 0.8669
Epoch 8/100
1/1 [==============================] - 0s 29ms/step - loss: 1.6704 - auc: 0.8767
Epoch 9/100
1/1 [==============================] - 0s 34ms/step - loss: 1.5591 - auc: 0.8825
Epoch 10/100
1/1 [==============================] - 0s 31ms/step - loss: 1.4109 - auc: 0.8918
Epoch 11/100
1/1 [==============================] - 0s 33ms/step - l

1/1 [==============================] - 0s 25ms/step - loss: 0.2943 - auc: 0.9845
Epoch 89/100
1/1 [==============================] - 0s 29ms/step - loss: 0.2887 - auc: 0.9859
Epoch 90/100
1/1 [==============================] - 0s 29ms/step - loss: 0.2794 - auc: 0.9871
Epoch 91/100
1/1 [==============================] - 0s 24ms/step - loss: 0.2861 - auc: 0.9854
Epoch 92/100
1/1 [==============================] - 0s 28ms/step - loss: 0.2827 - auc: 0.9861
Epoch 93/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2801 - auc: 0.9864
Epoch 94/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2835 - auc: 0.9857
Epoch 95/100
1/1 [==============================] - 0s 24ms/step - loss: 0.2678 - auc: 0.9898
Epoch 96/100
1/1 [==============================] - 0s 23ms/step - loss: 0.2827 - auc: 0.9855
Epoch 97/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2888 - auc: 0.9849
Epoch 98/100
1/1 [==============================] - 0s 25ms/step - loss: 

In [5]:
m='remove_Connectivity'
a=['AtomPairFP','Charge',
       'Kappa','MAP4','MOE','MorganFP','Topology']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100
1/1 [==============================] - 0s 339ms/step - loss: 2.7401 - auc: 0.2851
Epoch 2/100
1/1 [==============================] - 0s 27ms/step - loss: 1.6472 - auc: 0.6962
Epoch 3/100
1/1 [==============================] - 0s 34ms/step - loss: 1.4923 - auc: 0.8466
Epoch 4/100
1/1 [==============================] - 0s 34ms/step - loss: 1.6183 - auc: 0.8672
Epoch 5/100
1/1 [==============================] - 0s 27ms/step - loss: 1.7134 - auc: 0.8673
Epoch 6/100
1/1 [==============================] - 0s 31ms/step - loss: 1.7128 - auc: 0.8767
Epoch 7/100
1/1 [==============================] - 0s 37ms/step - loss: 1.6721 - auc: 0.8826
Epoch 8/100
1/1 [==============================] - 0s 30ms/step - loss: 1.5690 - auc: 0.8920
Epoch 9/100
1/1 [==============================] - 0s 35ms/step - loss: 1.4419 - auc: 0.8996
Epoch 10/100
1/1 [==============================] - 0s 26ms/step - loss: 1.2818 - auc: 0.9076
Epoch 11/100
1/1 [==============================] - 0s 28ms/step - l

1/1 [==============================] - 0s 27ms/step - loss: 0.2539 - auc: 0.9898
Epoch 89/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2627 - auc: 0.9880
Epoch 90/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2476 - auc: 0.9908
Epoch 91/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2567 - auc: 0.9888
Epoch 92/100
1/1 [==============================] - 0s 28ms/step - loss: 0.2445 - auc: 0.9911
Epoch 93/100
1/1 [==============================] - 0s 31ms/step - loss: 0.2432 - auc: 0.9912
Epoch 94/100
1/1 [==============================] - 0s 22ms/step - loss: 0.2422 - auc: 0.9915
Epoch 95/100
1/1 [==============================] - 0s 23ms/step - loss: 0.2513 - auc: 0.9897
Epoch 96/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2471 - auc: 0.9899
Epoch 97/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2359 - auc: 0.9919
Epoch 98/100
1/1 [==============================] - 0s 28ms/step - loss: 

In [6]:
m='remove_Charge'
a=['AtomPairFP','Connectivity',
       'Kappa','MAP4','MOE','MorganFP','Topology']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100
1/1 [==============================] - 0s 340ms/step - loss: 1.7207 - auc: 0.7258
Epoch 2/100
1/1 [==============================] - 0s 27ms/step - loss: 1.6243 - auc: 0.8077
Epoch 3/100
1/1 [==============================] - 0s 30ms/step - loss: 1.4703 - auc: 0.8153
Epoch 4/100
1/1 [==============================] - 0s 27ms/step - loss: 1.3509 - auc: 0.8223
Epoch 5/100
1/1 [==============================] - 0s 33ms/step - loss: 1.2346 - auc: 0.8641
Epoch 6/100
1/1 [==============================] - 0s 31ms/step - loss: 1.1270 - auc: 0.9056
Epoch 7/100
1/1 [==============================] - 0s 36ms/step - loss: 1.0783 - auc: 0.9201
Epoch 8/100
1/1 [==============================] - 0s 25ms/step - loss: 0.9964 - auc: 0.9420
Epoch 9/100
1/1 [==============================] - 0s 32ms/step - loss: 0.9215 - auc: 0.9563
Epoch 10/100
1/1 [==============================] - 0s 29ms/step - loss: 0.8795 - auc: 0.9606
Epoch 11/100
1/1 [==============================] - 0s 27ms/step - l

1/1 [==============================] - 0s 25ms/step - loss: 0.2154 - auc: 0.9931
Epoch 89/100
1/1 [==============================] - 0s 23ms/step - loss: 0.2244 - auc: 0.9914
Epoch 90/100
1/1 [==============================] - 0s 29ms/step - loss: 0.2203 - auc: 0.9923
Epoch 91/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2168 - auc: 0.9931
Epoch 92/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2189 - auc: 0.9924
Epoch 93/100
1/1 [==============================] - 0s 24ms/step - loss: 0.2169 - auc: 0.9931
Epoch 94/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2278 - auc: 0.9903
Epoch 95/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2168 - auc: 0.9926
Epoch 96/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2213 - auc: 0.9914
Epoch 97/100
1/1 [==============================] - 0s 23ms/step - loss: 0.2230 - auc: 0.9915
Epoch 98/100
1/1 [==============================] - 0s 23ms/step - loss: 

In [7]:
m='remove_Kappa'
a=['AtomPairFP','Connectivity',
       'Charge','MAP4','MOE','MorganFP','Topology']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100
1/1 [==============================] - 0s 341ms/step - loss: 1.8640 - auc: 0.5719
Epoch 2/100
1/1 [==============================] - 0s 28ms/step - loss: 1.6733 - auc: 0.7979
Epoch 3/100
1/1 [==============================] - 0s 26ms/step - loss: 1.6794 - auc: 0.8245
Epoch 4/100
1/1 [==============================] - 0s 34ms/step - loss: 1.5798 - auc: 0.8476
Epoch 5/100
1/1 [==============================] - 0s 30ms/step - loss: 1.4288 - auc: 0.8587
Epoch 6/100
1/1 [==============================] - 0s 35ms/step - loss: 1.3095 - auc: 0.8607
Epoch 7/100
1/1 [==============================] - 0s 27ms/step - loss: 1.2114 - auc: 0.8672
Epoch 8/100
1/1 [==============================] - 0s 33ms/step - loss: 1.1634 - auc: 0.8704
Epoch 9/100
1/1 [==============================] - 0s 28ms/step - loss: 1.1194 - auc: 0.8757
Epoch 10/100
1/1 [==============================] - 0s 27ms/step - loss: 1.0366 - auc: 0.9063
Epoch 11/100
1/1 [==============================] - 0s 28ms/step - l

1/1 [==============================] - 0s 29ms/step - loss: 0.2268 - auc: 0.9919
Epoch 89/100
1/1 [==============================] - 0s 24ms/step - loss: 0.2280 - auc: 0.9921
Epoch 90/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2287 - auc: 0.9916
Epoch 91/100
1/1 [==============================] - 0s 24ms/step - loss: 0.2345 - auc: 0.9904
Epoch 92/100
1/1 [==============================] - 0s 28ms/step - loss: 0.2257 - auc: 0.9921
Epoch 93/100
1/1 [==============================] - 0s 23ms/step - loss: 0.2224 - auc: 0.9926
Epoch 94/100
1/1 [==============================] - 0s 24ms/step - loss: 0.2271 - auc: 0.9918
Epoch 95/100
1/1 [==============================] - 0s 28ms/step - loss: 0.2272 - auc: 0.9917
Epoch 96/100
1/1 [==============================] - 0s 23ms/step - loss: 0.2333 - auc: 0.9899
Epoch 97/100
1/1 [==============================] - 0s 30ms/step - loss: 0.2258 - auc: 0.9910
Epoch 98/100
1/1 [==============================] - 0s 29ms/step - loss: 

In [8]:
m='remove_MAP4'
a=['AtomPairFP','Connectivity',
       'Charge','Kappa','MOE','MorganFP','Topology']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100
1/1 [==============================] - 1s 563ms/step - loss: 4.1729 - auc: 0.2322
Epoch 2/100
1/1 [==============================] - 0s 29ms/step - loss: 2.3875 - auc: 0.3626
Epoch 3/100
1/1 [==============================] - 0s 34ms/step - loss: 1.5298 - auc: 0.7778
Epoch 4/100
1/1 [==============================] - 0s 27ms/step - loss: 1.4723 - auc: 0.8688
Epoch 5/100
1/1 [==============================] - 0s 27ms/step - loss: 1.6602 - auc: 0.8762
Epoch 6/100
1/1 [==============================] - 0s 34ms/step - loss: 1.7979 - auc: 0.8755
Epoch 7/100
1/1 [==============================] - 0s 30ms/step - loss: 1.9113 - auc: 0.8659
Epoch 8/100
1/1 [==============================] - 0s 34ms/step - loss: 1.9134 - auc: 0.8689
Epoch 9/100
1/1 [==============================] - 0s 34ms/step - loss: 1.8811 - auc: 0.8714
Epoch 10/100
1/1 [==============================] - 0s 49ms/step - loss: 1.8212 - auc: 0.8686
Epoch 11/100
1/1 [==============================] - 0s 39ms/step - l

1/1 [==============================] - 0s 27ms/step - loss: 0.3191 - auc: 0.9826
Epoch 89/100
1/1 [==============================] - 0s 33ms/step - loss: 0.3114 - auc: 0.9839
Epoch 90/100
1/1 [==============================] - 0s 26ms/step - loss: 0.3090 - auc: 0.9848
Epoch 91/100
1/1 [==============================] - 0s 28ms/step - loss: 0.3154 - auc: 0.9824
Epoch 92/100
1/1 [==============================] - 0s 27ms/step - loss: 0.3095 - auc: 0.9835
Epoch 93/100
1/1 [==============================] - 0s 26ms/step - loss: 0.3057 - auc: 0.9842
Epoch 94/100
1/1 [==============================] - 0s 22ms/step - loss: 0.3047 - auc: 0.9838
Epoch 95/100
1/1 [==============================] - 0s 25ms/step - loss: 0.3064 - auc: 0.9832
Epoch 96/100
1/1 [==============================] - 0s 31ms/step - loss: 0.3042 - auc: 0.9836
Epoch 97/100
1/1 [==============================] - 0s 36ms/step - loss: 0.3042 - auc: 0.9831
Epoch 98/100
1/1 [==============================] - 0s 41ms/step - loss: 

In [9]:
m='remove_MOE'
a=['AtomPairFP','Connectivity',
       'Charge','Kappa','MAP4','MorganFP','Topology']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100
1/1 [==============================] - 0s 413ms/step - loss: 2.9898 - auc: 0.2490
Epoch 2/100
1/1 [==============================] - 0s 28ms/step - loss: 1.7109 - auc: 0.6384
Epoch 3/100
1/1 [==============================] - 0s 38ms/step - loss: 1.4778 - auc: 0.8404
Epoch 4/100
1/1 [==============================] - 0s 40ms/step - loss: 1.5993 - auc: 0.8610
Epoch 5/100
1/1 [==============================] - 0s 43ms/step - loss: 1.7401 - auc: 0.8623
Epoch 6/100
1/1 [==============================] - 0s 30ms/step - loss: 1.7527 - auc: 0.8727
Epoch 7/100
1/1 [==============================] - 0s 34ms/step - loss: 1.7191 - auc: 0.8762
Epoch 8/100
1/1 [==============================] - 0s 30ms/step - loss: 1.6626 - auc: 0.8760
Epoch 9/100
1/1 [==============================] - 0s 31ms/step - loss: 1.5319 - auc: 0.8881
Epoch 10/100
1/1 [==============================] - 0s 31ms/step - loss: 1.3692 - auc: 0.9005
Epoch 11/100
1/1 [==============================] - 0s 36ms/step - l

1/1 [==============================] - 0s 24ms/step - loss: 0.2704 - auc: 0.9887
Epoch 89/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2742 - auc: 0.9884
Epoch 90/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2594 - auc: 0.9908
Epoch 91/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2694 - auc: 0.9882
Epoch 92/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2571 - auc: 0.9912
Epoch 93/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2698 - auc: 0.9876
Epoch 94/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2629 - auc: 0.9898
Epoch 95/100
1/1 [==============================] - 0s 28ms/step - loss: 0.2754 - auc: 0.9868
Epoch 96/100
1/1 [==============================] - 0s 37ms/step - loss: 0.2607 - auc: 0.9892
Epoch 97/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2574 - auc: 0.9900
Epoch 98/100
1/1 [==============================] - 0s 26ms/step - loss: 

In [10]:
m='remove_MorganFP'
a=['AtomPairFP','Connectivity',
       'Charge','Kappa','MAP4','MOE','Topology']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100
1/1 [==============================] - 0s 338ms/step - loss: 2.5322 - auc: 0.3011
Epoch 2/100
1/1 [==============================] - 0s 26ms/step - loss: 1.6312 - auc: 0.7189
Epoch 3/100
1/1 [==============================] - 0s 30ms/step - loss: 1.6009 - auc: 0.8290
Epoch 4/100
1/1 [==============================] - 0s 33ms/step - loss: 1.6823 - auc: 0.8542
Epoch 5/100
1/1 [==============================] - 0s 32ms/step - loss: 1.7453 - auc: 0.8574
Epoch 6/100
1/1 [==============================] - 0s 32ms/step - loss: 1.7304 - auc: 0.8658
Epoch 7/100
1/1 [==============================] - 0s 28ms/step - loss: 1.6307 - auc: 0.8757
Epoch 8/100
1/1 [==============================] - 0s 32ms/step - loss: 1.4862 - auc: 0.8853
Epoch 9/100
1/1 [==============================] - 0s 33ms/step - loss: 1.3395 - auc: 0.8917
Epoch 10/100
1/1 [==============================] - 0s 35ms/step - loss: 1.2136 - auc: 0.8955
Epoch 11/100
1/1 [==============================] - 0s 28ms/step - l

1/1 [==============================] - 0s 28ms/step - loss: 0.2736 - auc: 0.9878
Epoch 89/100
1/1 [==============================] - 0s 28ms/step - loss: 0.2749 - auc: 0.9871
Epoch 90/100
1/1 [==============================] - 0s 24ms/step - loss: 0.2774 - auc: 0.9869
Epoch 91/100
1/1 [==============================] - 0s 24ms/step - loss: 0.2754 - auc: 0.9868
Epoch 92/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2690 - auc: 0.9887
Epoch 93/100
1/1 [==============================] - 0s 28ms/step - loss: 0.2612 - auc: 0.9895
Epoch 94/100
1/1 [==============================] - 0s 24ms/step - loss: 0.2599 - auc: 0.9896
Epoch 95/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2717 - auc: 0.9872
Epoch 96/100
1/1 [==============================] - 0s 24ms/step - loss: 0.2724 - auc: 0.9874
Epoch 97/100
1/1 [==============================] - 0s 24ms/step - loss: 0.2661 - auc: 0.9878
Epoch 98/100
1/1 [==============================] - 0s 28ms/step - loss: 

In [11]:
m='remove_Topology'
a=['AtomPairFP','Connectivity',
       'Charge','Kappa','MAP4','MOE','MorganFP']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100
1/1 [==============================] - 1s 521ms/step - loss: 2.4450 - auc: 0.3135
Epoch 2/100
1/1 [==============================] - 0s 31ms/step - loss: 1.6287 - auc: 0.7329
Epoch 3/100
1/1 [==============================] - 0s 29ms/step - loss: 1.5958 - auc: 0.8343
Epoch 4/100
1/1 [==============================] - 0s 33ms/step - loss: 1.6981 - auc: 0.8527
Epoch 5/100
1/1 [==============================] - 0s 36ms/step - loss: 1.7167 - auc: 0.8655
Epoch 6/100
1/1 [==============================] - 0s 29ms/step - loss: 1.6557 - auc: 0.8746
Epoch 7/100
1/1 [==============================] - 0s 35ms/step - loss: 1.5809 - auc: 0.8782
Epoch 8/100
1/1 [==============================] - 0s 32ms/step - loss: 1.4038 - auc: 0.8920
Epoch 9/100
1/1 [==============================] - 0s 34ms/step - loss: 1.2788 - auc: 0.8962
Epoch 10/100
1/1 [==============================] - 0s 33ms/step - loss: 1.1275 - auc: 0.9121
Epoch 11/100
1/1 [==============================] - 0s 29ms/step - l

1/1 [==============================] - 0s 27ms/step - loss: 0.2465 - auc: 0.9896
Epoch 89/100
1/1 [==============================] - 0s 29ms/step - loss: 0.2512 - auc: 0.9883
Epoch 90/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2303 - auc: 0.9930
Epoch 91/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2376 - auc: 0.9914
Epoch 92/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2440 - auc: 0.9901
Epoch 93/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2383 - auc: 0.9906
Epoch 94/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2415 - auc: 0.9900
Epoch 95/100
1/1 [==============================] - 0s 23ms/step - loss: 0.2371 - auc: 0.9913
Epoch 96/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2324 - auc: 0.9919
Epoch 97/100
1/1 [==============================] - 0s 23ms/step - loss: 0.2385 - auc: 0.9903
Epoch 98/100
1/1 [==============================] - 0s 29ms/step - loss: 

In [12]:
de=pd.DataFrame()
de['model']=model
de['auc']=AUC_ROC
de.to_csv('8feature_remove_one2.csv',index=False)
print(de)

                 model       auc
0            all_model  0.947846
1    remove_AtomPairFP  0.927867
2  remove_Connectivity  0.948833
3        remove_Charge  0.948729
4         remove_Kappa  0.947996
5          remove_MAP4  0.944140
6           remove_MOE  0.946867
7      remove_MorganFP  0.948022
8      remove_Topology  0.948303
